In [1]:
from alpaca_farm.models import reward_model
from alpaca_farm.inference.decode import load_model_and_tokenizer_for_inference 
from alpaca_farm.inference.score import score_sequences_with_huggingface_given_model
from alpaca_farm import utils
import alpaca_farm.data_preprocessor as data_preprocessor


import torch
from datasets import load_dataset
import pandas as pd
import numpy as np
from src.tfr_decoding.custom_bs import beam_search
from src.tfr_decoding.recurse_samp import sample
from src.utils.ap_samp_utils import inpsampall, dset_randsamp, sampfrominp, exhaustive_samp

device = 'cuda:0' # if you have a GPU

/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda120.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 120
CUDA SETUP: Loading binary /home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda120.so...


/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/prasann/miniconda3/envs/aplora did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/prasann/miniconda3/envs/aplora/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


ImportError: cannot import name 'inpsampall' from 'src.utils.ap_samp_utils' (/data/users/prasann/Projects/tfr-decoding/src/utils/ap_samp_utils.py)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
elidf = pd.read_json("output/elidataset.jsonl", orient="records", lines="true")
# 16000 for P1
# 25000 for P2
# 31500 for P3
# 50000 for P4
# 65000 for P5
# 70000 for P6
elidf = elidf.drop_duplicates(subset="history").iloc[15000:15050]

In [5]:
rew_model, tokenizer = load_model_and_tokenizer_for_inference(
    model_name_or_path="/home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/",
    model_cls=reward_model.RewardModel,
    cache_dir=None,
    model_kwargs=dict(
        torch_dtype=utils.convert_str_dtype_to_torch_dtype(None),
        flash_attn=False,
    ),
)

Loading model for inference: /home/prasann/Projects/tfr-decoding/apfarm_models/reward-model-human/
One of the `_name_or_path` to the root node is not a pretrained model name or local path on disk. This may be due to copying checkpoints across machines. Falling back to hardcoded rule to figure out the pretrained model based on config.json.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.63s/it]


In [6]:
rew_model.device

device(type='cuda', index=0)

In [7]:
torch.cuda.empty_cache()

In [8]:
model, tokenizer = load_model_and_tokenizer_for_inference(
    model_name_or_path='/home/prasann/Projects/tfr-decoding/apfarm_models/ppo-human/',
    cache_dir=None,
    model_kwargs=dict(torch_dtype=utils.convert_str_dtype_to_torch_dtype(None)),
)

Loading model for inference: /home/prasann/Projects/tfr-decoding/apfarm_models/ppo-human/
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.62s/it]


In [9]:
def generate_stuff(question):
    pdict_path = "/home/prasann/Projects/tfr-decoding/alpaca_farm/examples/prompts/v0_inputs_noinputs.json"
    prompts, list_dict_data, metadata = data_preprocessor.format_prompt_with_data_frame(
        df=pd.DataFrame({'instruction':[question], 'input':[""]}),
        prompt_dict=utils.jload(pdict_path),
    )
    inps = tokenizer(prompts, return_tensors="pt").to("cuda")
    outs = model.generate(inps.input_ids, attention_mask=inps.attention_mask, do_sample=True, temperature=1.0,max_new_tokens=300,num_return_sequences=2,)
    return tokenizer.batch_decode(outs, skip_special_tokens=True)

In [ ]:
#model.sample = sample.__get__(model)
model.eval()

In [11]:
instrs = []
pdict_path = "/home/prasann/Projects/tfr-decoding/alpaca_farm/examples/prompts/v0_inputs_noinputs.json"
for i, r in elidf.iterrows():
    prompts, list_dict_data, metadata = data_preprocessor.format_prompt_with_data_frame(
        df=pd.DataFrame({'instruction':[r['history']], 'input':[""]}),
        prompt_dict=utils.jload(pdict_path),
    )
    instrs.append(prompts[0])


In [12]:
elidf['history'] = instrs

In [15]:
prompt, orig_os, orig_scos, nav, oav, obests, oavgs, nb, ob, stats = sampfrominp(elidf, tokenizer, model, tokenizer, rew_model, 0, 0,  None, [0.5], [1], [1], 0.9)

Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None


cuda:0


evaluating rewards for batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.55it/s]

[180]
prefcut  90
PREFIX: 
['<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response:Since the start of the COVID-19 pandemic, Syria has been facing a major crisis, leading to protest']
newbest  0.7613333463668823 ; oldbest  0.7613333463668823
newavg  0.7613333463668823 ; oldavg 0.7613333463668823


In [ ]:
isall = inpsampall(elidf.iloc[:1], tokenizer, model, steamtok, steamshp, None, pflen, rchoose, tsamps, 0, 0.9)

In [23]:
len(tokenizer(orig_os[0]).input_ids[0])

266

In [32]:
exhaustive_samp(elidf.iloc[:1], tokenizer, model, tokenizer, rew_model, [4, 4], [4, 4], 1)

Inputs: 
['<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response:']


Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None


Outputs: 
 ['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response:Since 2011, the people of Syria have been protesting against the regime of President Bashar al-Assad, who has been in power since 2000. The protests began as demonstrations against the authoritarian government, but quickly escalated into a civil war as the Syrian military responded with violence against the protesters. Since then, the conflict has gotten much larger, involving various rebel groups, jihadist factions, and other foreign powers such as Russia and the United States. The Syrian Civil War has resulted in the deaths of hundreds of thousands of people, and has caused a massive humanitarian crisis, with millions of Syrians displaced from their homes and in need of aid. It is currently an ongoing confl

Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None


cuda:0


...

Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None


cuda:0


Unknown mixed precision mode: None, falling back to fp32.
mixed_precision = None


cuda:0
Inputs: 
['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response:Since 2011, the people of Syria have been protesting against the regime of President', 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response:Since 2011, the people of Syria have been protesting against the regime of President', 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSyrian Protests and Bashar al-AssadIm constantly hearing information about this, but i really have no idea what is going on. \n\n### Response


KeyboardInterrupt

